In [5]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

In [3]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
best_path_skim_dir = params['best_path_skim_dir']

perf_measure_columns = params['final_columns']

period = params['periods']

In [6]:
df_trips = create_trip_roster(ctramp_dir, hh, pp_perc ,transbay_od, geo_cwks, link21_purp_mapping, iteration)

NAs in PP Share: 0


In [7]:
df_trips['Period'] = df_trips['depart_hour'].map(time_period_mapping)
df_trips['Mode'] = df_trips['trip_mode'].map(mode_cat_mapping)

In [8]:
df_trn = df_trips.loc[df_trips['trip_mode'].isin([6,7,8])]

In [ ]:
#df_trn['trips'] = 1

In [9]:
def skim_core_to_df(skim, core, cols =['orig', 'dest', 'rail_od']):
    skim_df = pd.DataFrame(skim[core])
    skim_df = pd.melt(skim_df.reset_index(), id_vars='index', value_vars=skim_df.columns)
    skim_df['index'] = skim_df['index'] + 1
    skim_df['variable'] = skim_df['variable'] + 1
    skim_df.columns = cols

    return skim_df

In [10]:
def create_rail_crowding_od_pairs(transit_demand_dir, transit_skims_dir, period, acc_egg_modes):
    
    #Creates the Rail OD eligible Files
    for per in period:
        print("Period: ",per)

        rail_demand = omx.open_file(_join(transit_demand_dir, "rail_crowding_od_v9_trim_" + per.upper() + ".omx"),'w') 
        for acc_egg in acc_egg_modes:
            print("Access Egress Mode: ",acc_egg)
            trn_skm = omx.open_file(_join(transit_skims_dir, "trnskm" + per.lower() +"_" + acc_egg.upper() + ".omx"))
            crowd = np.array(trn_skm['CROWD']) * 1.62
            #rail_dmn = trn_dmn_acc * ivtrail
            rail_demand[acc_egg] = crowd

        rail_demand.close()

In [11]:
create_rail_crowding_od_pairs(transit_demand_dir, transit_skims_dir, period, acc_egg_modes)

Period:  am
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  md
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  pm
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ev
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ea
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR


In [13]:
df_temp = []

for period in time_periods:
    print(f'processing - {period}')
    
    df_od_pr = omx.open_file(_join(transit_demand_dir, "rail_od_v9_trim_" + period.upper() + ".omx"))
    df_od_cwd = omx.open_file(_join(transit_demand_dir, "rail_crowding_od_v9_trim_" + period.upper() + ".omx"))
    df_trn_pd = df_trn[df_trn['Period'] == period]
    
    #walk transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'WALK_TRANSIT']
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_wlk = pd.merge(df_trn_acc, df_rail_od, 
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    df_trn_wlk = pd.merge(df_trn_wlk, df_rail_cwd,
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    
    
    # PNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'PNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_PNR')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_PNR', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_pnr_inb = pd.merge(df_trn_pnr_inb, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'PNR_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'PNR_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_pnr_outbnd = pd.merge(df_trn_pnr_outbnd, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_pnr = pd.concat([df_trn_pnr_inb, df_trn_pnr_outbnd], ignore_index=True)
    
    # KNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'KNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'WLK_TRN_KNR', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_KNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_knr_inb = pd.merge(df_trn_knr_inb, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')


    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'KNR_TRN_WLK')
    df_rail_cwd = skim_core_to_df(df_od_cwd, 'KNR_TRN_WLK', cols =['orig', 'dest', 'crowd']) # add walk access/egress
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')
    df_trn_knr_outbnd = pd.merge(df_trn_knr_outbnd, df_rail_cwd, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_knr = pd.concat([df_trn_knr_inb, df_trn_knr_outbnd], ignore_index=True)
    
    df_trn_rail = pd.concat([df_trn_wlk, df_trn_pnr, df_trn_knr], ignore_index=True)
    df_temp.append(df_trn_rail)

df_trn_rail = pd.concat(df_temp)

processing - am
processing - md
processing - pm
processing - ev
processing - ea


In [ ]:
df_trn_rail

In [ ]:
df_trn_rail['trips'] = df_trn_rail['trips']*df_trn_rail['crowd']

In [ ]:
# summarise for prioirty population



#regional value
region_value = df_temp.groupby(['Period'])['trips'].mean().reset_index()
region_value = region_value.rename(columns={'trips': 'Value'})
region_value['Concept_ID'] = concept_id
region_value['Metric_ID'] = 'A1.9'
region_value['Metric_name'] = 'Crowding (Region)'
region_value['Submetric'] = 'A1.9.1'
region_value['Description'] = 'regional crowding level'
region_value['Population'] = 'Whole Population'
region_value['Geography'] = 'Regional'
region_value['Orig_zone'] = ''
region_value['Dest_zone'] = ''
region_value['Zone_ID'] = 'Megaregion'
region_value['Units'] = 'minutes'
region_value = region_value[perf_measure_columns]

#county 
county_df_temp = df_trn_rail.groupby(['orig_county', 'dest_county', 'Period'])['trips'].mean().reset_index()
county_df_temp = county_df_temp.rename(columns={'trips': 'Value',
                                               'orig_county': 'Origin_zone',
                                               'dest_county': 'Dest_zone'})
county_df_temp['Concept_ID'] = concept_id
county_df_temp['Metric_ID'] = 'A1.9'
county_df_temp['Metric_name'] = 'Crowding (Region)'
county_df_temp['Submetric'] = 'A1.9.2'
county_df_temp['Description'] = 'regional crowding level'
county_df_temp['Population'] = 'Whole Population'
county_df_temp['Geography'] = 'County'
county_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
county_df_temp['Units'] = 'minutes'
county_df_temp['Total_Increment'] = ''


#super district
sd_df_temp = df_trn_rail.groupby(['orig_super_dist', 'dest_super_dist', 'Period'])['trips'].mean().reset_index()
sd_df_temp = sd_df_temp.rename(columns={'trips': 'Value',
                                        'orig_super_dist': 'Origin_zone',
                                        'dest_super_dist': 'Dest_zone'})
sd_df_temp['Concept_ID'] = concept_id
sd_df_temp['Metric_ID'] = 'A1.9'
sd_df_temp['Metric_name'] = 'Crowding (Region)'
sd_df_temp['Submetric'] = 'A1.9.3'
sd_df_temp['Description'] = 'regional crowding level'
sd_df_temp['Population'] = 'Whole Population'
sd_df_temp['Geography'] = 'Super district'
sd_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
sd_df_temp['Units'] = 'minutes'
sd_df_temp['Total_Increment'] = ''


#RDM Zones
rdm_df_temp = df_trn_rail.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Period'])['trips'].mean().reset_index()
rdm_df_temp = rdm_df_temp.rename(columns={'trips': 'Value',
                                        'orig_rdm_zones': 'Origin_zone',
                                        'dest_rdm_zones': 'Dest_zone'})
rdm_df_temp['Concept_ID'] = concept_id
rdm_df_temp['Metric_ID'] = 'A1.9'
rdm_df_temp['Metric_name'] = 'Crowding (Region)'
rdm_df_temp['Submetric'] = 'A1.9.4'
rdm_df_temp['Description'] = 'regional crowding level'
rdm_df_temp['Population'] = 'Whole Population'
rdm_df_temp['Geography'] = 'RDM'
rdm_df_temp['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
rdm_df_temp['Units'] = 'minutes'
rdm_df_temp['Total_Increment'] = ''




# transbay region
tb_value = df_trn_rail[df_trn_rail['transbay_od']==1]
tb_value = tb_value.groupby(['Period'])['trips'].mean().reset_index()

tb_value['Concept_ID'] = concept_id
tb_value['Metric_ID'] = 'A1.9'
tb_value['Metric_name'] = 'Crowding (Region)'
tb_value['Submetric'] = 'A1.9.5'
tb_value['Description'] = 'regional crowding level'
tb_value['Population'] = 'Whole Population'
tb_value['Geography'] = 'Transbay'
tb_value['Zone_ID'] = ''
tb_value['Origin_zone'] = ''
tb_value['Dest_zone'] = ''
tb_value['Units'] = 'minutes'
tb_value['Total_Increment'] = ''